# Gradient boosting

**Gradient boosting** es una técnica genérica que interpreta FSAM como un algoritmo descenso por gradiente para un problema de minimización en un espacio funcional:
$$\hat{\boldsymbol{f}}%
=\operatorname*{argmin}_{\boldsymbol{f}}\mathcal{L}(\boldsymbol{f})
=\operatorname*{argmin}_{\boldsymbol{f}}\sum_{i=1}^N \ell(y_i,f(\boldsymbol{x}_i))$$
donde, por simplicidad, las funciones se representan por sus valores en el conjunto de entrenamiento, $\boldsymbol{f}=(f(\boldsymbol{x}_1),\dotsc,f(\boldsymbol{x}_N))^t$.

## Iteración $m$

En la iteración $m$ se desea construir un modelo $\boldsymbol{f}_m$ añadiendo una función base, $F_m$, a un modelo previo, $\boldsymbol{f}_{m-1}$, tal que $\mathcal{L}(\boldsymbol{f}_m)\leq\mathcal{L}(\boldsymbol{f}_{m-1})$. Visto como un problema de minimización en un espacio funcional, descenso por gradiente consiste en escoger la "dirección" de máximo descenso, esto es, la del negativo del gradiente de $\mathcal{L}(\boldsymbol{f})$ en $\boldsymbol{f}_{m-1}$, $\boldsymbol{g}_m$, y construir el nuevo modelo como:
$$\boldsymbol{f}_m=\boldsymbol{f}_{m-1}-\beta_m \boldsymbol{g}_m
\qquad\text{con}\qquad%
g_{im}=\left[\frac{\partial\ell(y_i,f(\boldsymbol{x}_i))}{\partial f(\boldsymbol{x}_i)}\right]_{f_{m-1}(\boldsymbol{x}_i)}$$
donde el factor de aprendizaje puede escogerse por búsqueda lineal. Ahora bien, el algoritmo no es muy útil si se limita a funciones representadas por sus valores en el conjunto de entrenamiento, ya que no podemos generalizar. Para poder hacerlo, gradient boosting ajusta el aprendiz débil al negativo del gradiente:
$$F_m=\operatorname*{argmin}_F\sum_{i=1}^N(-g_{im}-F(\boldsymbol{x}_i))^2$$

## Algoritmo básico

El algoritmo básico prescinde de $\beta_m$ pero incluye un **shrinkage factor** $0<\nu\leq 1$ para facilitar la regularización:
1. Inicializar $f_0(\boldsymbol{x})=\operatorname{argmin}_F\sum_{i=1}^N\ell(y_i,F(\boldsymbol{x}_i))$
2. **for** $\;m=1:M\;$ **do**
3. $\quad$ Calcular el negativo del gradiente o (pseudo-)residuo $r_{im}=-\left[\dfrac{\partial\ell(y_i,f(\boldsymbol{x}_i))}{\partial f(\boldsymbol{x}_i)}\right]_{f_{m-1}(\boldsymbol{x}_i)}$
4. $\quad$ Usar el aprendiz débil para hallar $F_m=\operatorname*{argmin}_F\sum_{i=1}^N(r_{im}-F(\boldsymbol{x}_i))^2$
5. $\quad$ Actualizar $f_m(\boldsymbol{x})=f_{m-1}(\boldsymbol{x})+\nu F_m(\boldsymbol{x})$
6. Devolver $f(\boldsymbol{x})=f_M(\boldsymbol{x})$

## Pérdidas y neg-gradientes para regresión

En regresión, $y_i\in\mathbb{R}$ y se suele usar el error cuadrático como función de pérdida. Más concretamente, se usa la mitad del error cuadrático:
$$\ell(y_i,f(\boldsymbol{x}_i))=\frac{1}{2}(y_i-f(\boldsymbol{x}_i))^2$$
pues el neg-gradiente es el residuo de la predicción:
$$r_i=y_i-f(\boldsymbol{x}_i)$$
Nótese que gradient boosting con pérdida cuadrática coincide con boosting mínimos cuadrados.

Una alternativa al error cuadrático en regresión es el error absoluto:
$$\ell(y_i,f(\boldsymbol{x}_i))=\lvert y_i-f(\boldsymbol{x}_i)\rvert$$
El neg-gradiente es el signo del residuo de la predicción:
$$r_i=\operatorname{sgn}(y_i-f(\boldsymbol{x}_i))$$

## Pérdidas y neg-gradientes para clasificación binaria

En clasificación binaria, $\tilde{y}_i\in\{-1,+1\}$ y se suele usar la pérdida exponencial:
$$\ell(\tilde{y}_i,f(\boldsymbol{x}_i))=\exp(-\tilde{y}_i f(\boldsymbol{x}_i))$$
con
$$r_i=\tilde{y}_i\exp(-\tilde{y}_i f(\boldsymbol{x}_i))$$
Gradient boosting con pérdida exponencial es prácticamente idéntico a AdaBoost.

También se usa la log-pérdida binaria:
$$\ell(\tilde{y}_i,f(\boldsymbol{x}_i))=\log(1+\exp(-\tilde{y}_i f(\boldsymbol{x}_i)))$$
con
$$\begin{align*}
r_i%
&=-\frac{1}{1+\exp(-\tilde{y}_i f(\boldsymbol{x}_i))}\exp(-\tilde{y}_i f(\boldsymbol{x}_i))(-\tilde{y}_i)\\
&=\tilde{y}_i\frac{1}{1+\exp(\tilde{y}_i f(\boldsymbol{x}_i))}\\
&=\tilde{y}_i\sigma(-\tilde{y}_i f(\boldsymbol{x}_i))
\end{align*}$$
Gradient boosting con log-pérdida binaria es prácticamente idéntico a LogitBoost; algo más sencillo de implementar.

## Pérdidas y neg-gradientes para clasificación multiclase

En clasificación multiclase, $y_i\in\{1,\dotsc,C\}$ y se suele usar la log-pérdida. El algoritmo básico se extiende para ajustar $C$ modelos aditivos, uno por cada clase, cuyas predicciones se normalizan mediante una softmax:
$$\ell(y_i,f_1(\boldsymbol{x}_i),\dotsc,f_C(\boldsymbol{x}_i))=-\sum_c \mathbb{I}(y_i=c)\log \pi_{ic}$$
donde
$$\pi_{ic}=S(f_1(\boldsymbol{x}_i),\dotsc,f_C(\boldsymbol{x}_i))_c%
=\frac{\exp(f_c(\boldsymbol{x}_i))}{\sum_{c'=1}^C \exp(f_{c'}(\boldsymbol{x}_i))}$$
El neg-gradiente para la clase $c$ es:
$$\begin{align*}
r_{ic}%
&=-\dfrac{\partial\ell(y_i,f_1(\boldsymbol{x}_i),\dotsc,f_C(\boldsymbol{x}_i))}{\partial f_c(\boldsymbol{x}_i)}\\%
&=\frac{\partial}{\partial f_c(\boldsymbol{x}_i)}\sum_{\tilde{c}} \mathbb{I}(y_i=\tilde{c})\log \pi_{i\tilde{c}}\\%
&=\mathbb{I}(y_i=c)\frac{1}{\pi_{ic}}\pi_{ic}(1-\pi_{ic})\\%
&=\mathbb{I}(y_i=c)(1-\pi_{ic})
\end{align*}$$

## Gradient tree boosting

**Gradient tree boosting** es gradient boosting con árbol de regresión como aprendiz débil:
$$F_m=\sum_{j=1}^{J_m}w_{jm}\mathbb{I}(\boldsymbol{x}\in R_{jm})$$
donde $R_{jm}$ y $w_{jm}$ son la región y salida asociadas a la hoja $j$ del árbol añadido en la iteración $m$. La salida puede ser un escalar o, más generalmente, un vector (de probabilidades, por ejemplo).

El aprendizaje de las regiones se lleva a cabo mediante CART sobre residuos. La salida de cada hoja se aprende mediante minimización del riesgo empírico con los datos de la hoja:
$$\hat{w}_{jm}=\operatorname*{argmin}_w\sum_{\boldsymbol{x}_i\in R_{jm}}%
\ell(y_i,f_{m-1}(\boldsymbol{x}_i)+w)$$
En el caso de la pérdida cuadrática, $\hat{w}_{jm}$ es la media empírica de los residuos de la hoja.

## XGBoost

**Extreme gradient boosting (XGBoost)** es una implementación muy popular de gradient tree boosting con algunos refinamientos adicionales:
* Objetivo regularizado
* Aproximación de segundo orden de la pérdida
* Muestreo de caracterı́sticas en nodos internos
* Técnicas informáticas varias para mejorar la escabilidad